# Laboratorio No.2 Redes Neuronales

## Andrés Urízar - 17632

In [1]:
import mnist_reader
import numpy as np
from scipy import optimize as op
from functools import reduce
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')
#data = pd.read_csv('fashion-mnist_train.csv')

### Se procesa la información

In [10]:
X = X_train/1000.0  #Normalizacion de los datos
m,n = X.shape
y = y_train.reshape(m,1)
Y = (y == np.array(range(10))).astype(int)

In [11]:
X.shape

(60000, 784)

In [12]:
Y.shape

(60000, 10)

### Se comienzan a realizar las funciones

#### (Se deben correr las funciones para probarlo)

#### Función para dar forma a las matrices

In [6]:
flatten_list_of_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)    

In [7]:
def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes)+1
    sizes = [shape[0]*shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)
    
    for i in range(layers-1):
        steps[i+1]=steps[i]+sizes[i]
        
    return[
        flat_thetas[steps[i]: steps[i+1]].reshape(*shapes[i])
        for i in range(layers-1)
    ]

In [8]:
sigmoid = lambda z: 1.0/(1.0+np.exp(-z))

In [15]:
#Forward propagation (lista de thetas, X=matrix(60000,784)) se realiza paso 2.1 y 2.2
def feed_propagation(thetas, X):
    a = [X] #Se guarda en la primera posicion el input en este caso X (paso 2.1)
    for i in range(len(thetas)): #Paso 2.2 para obtencion de matrices de activacion
        a.append(
            sigmoid(
                np.matmul(
                    np.hstack((np.ones(len(X)).reshape(len(X),1),a[i])),
                    thetas[i].T
                )
            )
        )
    return a

In [16]:
#Back propagation, se realizan pasos 2.3 y 2.4
def back_propagation(activate,lay,result,theta):
    deltas = [*range(lay-1), activate[-1]-result] #2.3 se calcula el error de la ultima capa y se almacena
    #2.4 se realiza el ciclo desde la penultima capa hasta la capa antes del inicio
    for i in range(lay-2,0,-1):
        #print(((np.delete(theta[i],0,1)).T).shape)
        #print(deltas[i+1].shape)
        #print((activate[i]*(1-activate[i])).shape)
        #deltas[i] = np.matmul((np.delete(theta[i],0,1)).T, deltas[i+1])*(activate[i]*(1-activate[i])) v.1.0
        deltas[i] = np.matmul(deltas[i+1],(np.delete(theta[i],0,1)))*(activate[i]*(1-activate[i])) #v2.0
    return deltas

In [17]:
#Se declaran las Deltas mayuscular con la forma de las Thetas, se hace paso 1
def deltaDeclaration(shape):
    delta = []
    for sh in shape:
        temp = np.zeros(sh, dtype=int)
        delta.append(temp)
    return delta

In [18]:
#Funcion de costo
def cost_function(flat_thetas, shapes, X, Y):
    activation_array = feed_propagation(
        inflate_matrixes(flat_thetas, shapes),
        X
    )

    return -(Y * np.log(activation_array[-1]) + (1 - Y) * np.log(1 - activation_array[-1])).sum() / len(X) 

In [13]:
#Calculo de predicion que no funcionaaaaaaaaa
# def prediction_calculation(flat_thetas, shapes, X, Y):
#     #Delta = deltaDeclaration(shapes)
#     Delta = []
#     m, layers = len(X), len(theta_shapes)+1
#     thetas = inflate_matrixes(flat_thetas,theta_shapes)
#     activation_array = feed_propagation(flat_thetas,shapes)
#     deltas = back_propagation(activation_array,layers,Y,thetas)
#     #2.5
#     for i in range(layers-1):
#         Delta.append(
#             (deltas[i+1].T @ np.hstack((np.ones(len(activation_array[i]),1),activation_array[i])))/m
#         )
#         #Delta[i] = Delta[i] + np.matmul(deltas[i+1],(np.vstack((np.ones(len(X[0])).reshape(1,len(X[0])), activation_array[i]))).T)

In [84]:
def prediction_calculation(flat_thetas, shapes, X, Y):
    #Delta = deltaDeclaration(shapes)
        Deltaslist = []
        m, layers = len(X), len(shapes)+1
        thetas = inflate_matrixes(flat_thetas,shapes)
        activation_array = feed_propagation(thetas,X)
        #deltas = back_propagation(activation_array,layers,Y,thetas)
        
        
        deltas = [*range(layers-1), activation_array[-1]-Y] #2.3
    #2.4 se realiza el ciclo desde la penultima capa hasta la capa antes del inicio
        for i in range(layers-2,0,-1):
            deltas[i] = np.matmul(deltas[i+1],(np.delete(thetas[i],0,1)))*(activation_array[i]*(1-activation_array[i]))
            
        #2.5 v1.0
#         for i in range(layers-1):
#             a,b = thetas[i].shape
#             Delta = np.zeros(a*b).reshape(a,b)
#             x, _ = activation_array[i].shape
#             for j in range(x):
#                 Delta = Delta + np.matmul(((deltas[i+1][j]).reshape(len(deltas[i + 1][j]),1)),
#                 ((np.append([1],activation_array[i][j])).reshape(len(activation_array[i][j])+1,1)).T)
            
#             Deltas.append(Delta)
        
#         Deltas = np.asarray(Deltas)

#         return flatten_list_of_arrays([
#             np.true_divide(d,m)
#             for d in Deltas
#         ])
        # 2.5 v2.0
        for i in range(layers-1):
            Deltaslist.append(
                (
                    np.matmul(
                        deltas[i+1].T,
                        np.hstack((
                            np.ones(len(activation_array[i])).reshape(len(activation_array[i]),1),
                            activation_array[i])))
                )/m
            )
        Deltaslist = np.asarray(Deltaslist)
        
        return flatten_list_of_arrays(Deltaslist)

### Se realiza la arquitectura de las matrices de la red neuronal

In [13]:
REDES = np.array([
        n,
        125,
        10
    ])
# una sola capa de 125 neuronas
theta_shapes = np.hstack((
    REDES[1:].reshape(len(REDES)-1, 1),
    (REDES[:-1]+1).reshape(len(REDES)-1,1)
))


flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape)*0.01 
    for theta_shape in theta_shapes
])

### Desarrollo de red neuronal

In [86]:
print("\n---------------- OPTIMIZING ----------------\n")
result = op.minimize(
    fun = cost_function,
    x0 = flat_thetas,
    args = (theta_shapes, X, Y),
    method = 'L-BFGS-B',
    jac = prediction_calculation,
    options = {'disp': True, 'maxiter': 3000}
)
print("\n---------------- OPTIMIZED ----------------\n")



---------------- OPTIMIZING ----------------



<ipython-input-82-f3f1e6bf45b6>:8: RuntimeWarning: divide by zero encountered in log
  return -(Y * np.log(activation_array[-1]) + (1 - Y) * np.log(1 - activation_array[-1])).sum() / len(X)
<ipython-input-82-f3f1e6bf45b6>:8: RuntimeWarning: invalid value encountered in multiply
  return -(Y * np.log(activation_array[-1]) + (1 - Y) * np.log(1 - activation_array[-1])).sum() / len(X)



---------------- OPTIMIZED ----------------



In [161]:
# Se guarda el resultado de thetas optimizadas
np.savetxt('thetas_optimized3.txt', result.x,delimiter=',')

In [160]:
result.x.size

99385

In [144]:
result

      fun: nan
 hess_inv: <99385x99385 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.31761763e-08,  3.86480649e-15,  2.96099077e-15, ...,
       -1.00984083e-15,  1.93395179e-07, -5.79911462e-08])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 12103
      nit: 1687
   status: 0
  success: True
        x: array([-3.80816181e-01,  4.05849884e-03,  8.14810819e-03, ...,
        1.37269472e+00, -1.84523917e+01,  3.31388030e-01])

In [30]:
thetaResult = np.loadtxt('thetas_optimized3.txt') # se carga la theta de un txt
thetaResult.size

99385

### Test de thetas

In [31]:
X_test = X_test/1000
mT,nT = X_test.shape
y_test = y_test.reshape(mT,1)

In [32]:
flat_thetas_result = thetaResult

In [33]:
inflate_thetas_result = inflate_matrixes(flat_thetas_result, theta_shapes)
a = feed_propagation(inflate_thetas_result, X_test)

In [34]:
# Se obtienen la nuerona maxima de cada dato de test para obtener la prediccioni
maximos = np.argmax(a[-1], axis = 1).reshape(mT,1)
# Se compara con los resultados verdaderos
correct = ((maximos == y_test)*1).sum()
# Se saca el porcentaje de aciertos
porBuenos = (correct/mT)*100
# Y el de errores
porMalos = 100.0 - porBuenos

10.0

In [35]:
porBuenos

10.0

In [36]:
porMalos

90.0

In [29]:
pruebaDelta = deltaDeclaration(theta_shapes)
pruebaDelta[1].shape

(10, 126)

In [30]:
m, layers = len(X), len(theta_shapes)+1

In [31]:
thetas = inflate_matrixes(flat_thetas,theta_shapes)

In [32]:
pruebatran = thetas[0].T
thetas[0].shape

(125, 785)

In [17]:
a = feed_propagation(thetas, X)

NameError: name 'thetas' is not defined

In [35]:
a[-1][0,:]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [73]:
delta = back_propagation(a,layers,Y,thetas)
delta[2].shape

(60000, 10)